In [150]:
%%time
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Wall time: 951 µs


In [151]:
import numpy as np
import pandas as pd
import requests

In [152]:
date = 20200810
currencies_prices_url = f"https://www4.bcb.gov.br/Download/fechamento/{date}.csv"
currencies_info_url = f"https://www4.bcb.gov.br/Download/fechamento/{date}.csv"

cols = ['Timestamp', 'Cod Moeda', 'Tipo', 'Moeda', 'Taxa Compra', 'Taxa Venda', 'Paridade Compra', 'Paridade Venda'] 
df = pd.read_csv('data/20200810.csv', sep=";", names=cols) 
df.head()

,Timestamp,Cod Moeda,Tipo,Moeda,Taxa Compra,Taxa Venda,Paridade Compra,Paridade Venda
0,10/08/2020,5,A,AFN,"0,07004000","0,07041000","76,60000000","77,00000000"
1,10/08/2020,9,A,ETB,"0,14930000","0,15220000","35,44690000","36,13150000"
2,10/08/2020,15,A,THB,"0,17330000","0,17340000","31,11000000","31,12000000"
3,10/08/2020,20,A,PAB,"5,39270000","5,39330000","1,00000000","1,00000000"
4,10/08/2020,27,A,VES,"0,00001940","0,00001940","277343,55110000","278038,64770000"


In [153]:
# fixing the currenct type A and B formula by setting all rows referencing dollar

In [154]:
aux = df.set_index('Moeda')
usd = aux.loc['USD']
usd

Timestamp          10/08/2020
Cod Moeda                 220
Tipo                        A
Taxa Compra        5,39270000
Taxa Venda         5,39330000
Paridade Compra    1,00000000
Paridade Venda     1,00000000
Name: USD, dtype: object

In [155]:
usd_value = float(usd['Taxa Compra'].replace(',','.'))
usd_value

5.3927

In [156]:
# Casting strings to float
df["Taxa Compra"] = df["Taxa Compra"].str.replace(',','.')
df["Paridade Compra"] = df["Paridade Compra"].str.replace(',','.')
df.head(2)

,Timestamp,Cod Moeda,Tipo,Moeda,Taxa Compra,Taxa Venda,Paridade Compra,Paridade Venda
0,10/08/2020,5,A,AFN,0.07004000,"0,07041000",76.60000000,"77,00000000"
1,10/08/2020,9,A,ETB,0.14930000,"0,15220000",35.44690000,"36,13150000"


In [159]:
# new column with fixed formula
new_df = df.copy(deep=True)
new_df['Paridade Compra USD'] = usd_value / (new_df['Taxa Compra'].astype(float))

# validating
currencies = ["USD", "EUR", "CAD"]

In [160]:
# OLD/ORIGINAL DF
df.loc[new_df['Moeda'].isin(currencies)]

,Timestamp,Cod Moeda,Tipo,Moeda,Taxa Compra,Taxa Venda,Paridade Compra,Paridade Venda
34,10/08/2020,165,A,CAD,4.03860000,"4,03960000",1.33510000,"1,33530000"
47,10/08/2020,220,A,USD,5.39270000,"5,39330000",1.00000000,"1,00000000"
154,10/08/2020,978,B,EUR,6.33860000,"6,34140000",1.17540000,"1,17580000"


In [162]:
# NEW DF
new_df.loc[new_df['Moeda'].isin(currencies)]

,Timestamp,Cod Moeda,Tipo,Moeda,Taxa Compra,Taxa Venda,Paridade Compra,Paridade Venda,Paridade Compra USD
34,10/08/2020,165,A,CAD,4.03860000,"4,03960000",1.33510000,"1,33530000",1.335289
47,10/08/2020,220,A,USD,5.39270000,"5,39330000",1.00000000,"1,00000000",1.000000
154,10/08/2020,978,B,EUR,6.33860000,"6,34140000",1.17540000,"1,17580000",0.850771


In [165]:
# SORTING BY CURRENCY
new_df.sort_values(by='Paridade Compra USD', ascending=False)

,Timestamp,Cod Moeda,Tipo,Moeda,Taxa Compra,Taxa Venda,Paridade Compra,Paridade Venda,Paridade Compra USD
4,10/08/2020,27,A,VES,0.00001940,"0,00001940",277343.55110000,"278038,64770000",277974.226804
125,10/08/2020,815,A,IRR,0.00012840,"0,00012840",42000.00000000,"42000,00000000",41999.221184
53,10/08/2020,260,A,VND,0.00023260,"0,00023280",23170.00000000,"23185,00000000",23184.436801
30,10/08/2020,148,A,STD,0.00025590,"0,00025680",21000.00000000,"21070,00000000",21073.466198
137,10/08/2020,865,A,IDR,0.00036710,"0,00036970",14590.00000000,"14690,00000000",14690.002724
...,...,...,...,...,...,...,...,...,...
123,10/08/2020,805,A,OMR,14.00340000,"14,01220000",0.38490000,"0,38510000",0.385099
18,10/08/2020,105,A,BHD,14.30420000,"14,30580000",0.37700000,"0,37700000",0.377001
17,10/08/2020,100,A,KWD,17.60020000,"17,65980000",0.30540000,"0,30640000",0.306400
42,10/08/2020,197,B,CLF,193.66260000,"193,68420000",35.91200000,"35,91200000",0.027846
